In [ ]:
# Directory for your dataset
!pip install --upgrade --no-cache-dir gdown
from google.colab import files
uploaded = files.upload() # Upload cookies file
!mv cookies.txt ~/.cache/gdown/cookies.txt
# Step 2: Download the public Google Drive folder
!gdown --folder --remaining-ok "https://drive.google.com/drive/folders/12VOOehuYGuuRjQIH3VJy-XN4yWXzyYZG"


Saving cookies.txt to cookies (1).txt
Retrieving folder contents
Retrieving folder 1B3Y5KPfDxvW4ZUUBRY9SGyTeAeTpAsQ5 appDataSet
Retrieving folder 1iQ-xXN8Q4Pym-Aas3YrLFdfJj_rbZDkB  annotations
Processing file 1CF8BCYGaGGY7H0vNtn1LLIOTb34LOpTp IMG_5897.xml
Processing file 1g8xAf-5kNLyTJADOfPdSTJ2Kbm-UhB6q IMG_5898.xml
Processing file 1665xcUFdb83ySYnMouWU4dgJc_Dfdq5y IMG_5899.xml
Processing file 1vs-B2hA9Bjm5r3280aJ6kmGmTWQYlyQg IMG_5900.xml
Processing file 1kwYNd_FyyncH0hK1ScQ4oPFOO1LnXQ9f IMG_5901.xml
Processing file 1eTfrp99d0QiP_YONftBLDGILGUmtrJBq IMG_5902.xml
Processing file 1nhjylZnfc8ph4dru2PzCx7WWcYxIo1Au IMG_5903.xml
Processing file 1pP48CyOszORcXjimyj1LSeZdZT84tIBV IMG_5904.xml
Processing file 1lq3_zdaWy9RcMy6FdwwRoG7LPxHv7-Aw IMG_8505.xml
Processing file 1ZVx55pYzOZHeT-F2fkubRgfOPo4OOOzu IMG_8506.xml
Processing file 1b843C8BzZ__UtZ6TDO29V0NpU4Up74gH IMG_8507.xml
Processing file 1Lr7JvXZ21EnpG4Hh7AYFrh8mM9V6UYNd IMG_8508.xml
Processing file 1DwVm-FAz01GttanMygMkQF3o9IZCdYhJ I

In [ ]:
!pip install -U torch torchvision
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances

# Register the dataset (using the same COCO JSON for training and validation here)
register_coco_instances("climbing_holds_train", {}, COCO_JSON, IMAGES_DIR)
register_coco_instances("climbing_holds_val", {}, COCO_JSON, IMAGES_DIR)

# Get metadata and dataset dictionary for visualization
metadata = MetadataCatalog.get("climbing_holds_train")
dataset_dicts = DatasetCatalog.get("climbing_holds_train")

# Visualize a few samples (optional)
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    from detectron2.utils.visualizer import Visualizer
    v = Visualizer(img[:, :, ::-1], metadata=metadata)
    out = v.draw_dataset_dict(d)
    plt.figure(figsize=(10, 8))
    plt.imshow(out.get_image())
    plt.axis('off')
    plt.show()

# Configure the model for training
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("climbing_holds_train",)
cfg.DATASETS.TEST = ("climbing_holds_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Pretrained weights
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 5000    # Adjust iterations based on your dataset
cfg.SOLVER.STEPS = []         # No learning rate decay steps
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(metadata.thing_classes)  # Number of classes derived from your annotations
cfg.OUTPUT_DIR = os.path.join(BASE_DIR, "output_climbing")
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Train the model
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

NameError: name 'COCO_JSON' is not defined